In [ ]:
import pandas as pd
import matplotlib

df = pd.read_csv('agaricus-lepiota.data', header=None)

df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
5,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g
6,e,b,s,w,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,n,m
7,e,b,y,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,s,m
8,p,x,y,w,t,p,f,c,n,p,...,s,w,w,p,w,o,p,k,v,g
9,e,b,s,y,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,s,m


In [65]:
df = df.rename(columns={0: 'Poisonous', 5: 'Odor', 1: 'Cap Shape'})
df.head(10)

,Poisonous,Cap Shape,2,3,4,Odor,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
5,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g
6,e,b,s,w,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,n,m
7,e,b,y,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,s,m
8,p,x,y,w,t,p,f,c,n,p,...,s,w,w,p,w,o,p,k,v,g
9,e,b,s,y,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,s,m


In [66]:
subset = df[['Poisonous', 'Odor', 'Cap Shape']].copy()
subset.head(10)

,Poisonous,Odor,Cap Shape
0,p,p,x
1,e,a,x
2,e,l,b
3,p,p,x
4,e,n,x
5,e,a,x
6,e,a,b
7,e,l,b
8,p,p,x
9,e,a,b


In [67]:
poisonous = {'p': 1, 'e':0}
odor = {'a': 0, 'l': 1, 'c': 2, 'y': 3, 'f': 4, 'm': 5, 'n': 6, 'p': 7, 's': 8}
cap_shape = {'b': 0, 'c': 1, 'x': 2, 'f': 3, 'k': 4, 's': 5}


subset['Poisonous'] = subset['Poisonous'].map(poisonous)
subset['Odor'] = subset['Odor'].map(odor)
subset['Cap Shape'] = subset['Cap Shape'].map(cap_shape)
subset.head(10)

,Poisonous,Odor,Cap Shape
0,1,7,2
1,0,0,2
2,0,1,0
3,1,7,2
4,0,6,2
5,0,0,2
6,0,0,0
7,0,1,0
8,1,7,2
9,0,0,0


In [68]:
hist = subset.hist()

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.